In [1]:
import sys
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data.dataset import Dataset
from PIL import Image


In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [3]:
Epoch = 100

## Dataloader

In [4]:
datalist = []
facePath = './dataset/AFDB_face_dataset'
maskPath = './dataset/AFDB_masked_face_dataset'

## Get face data
for root, dirs, files in os.walk(facePath):
    if files == []:
        continue
    path1 = os.path.join(root, files[0])
    path2 = os.path.join(root, files[1])
    datalist.append([path1, 0])
    datalist.append([path2, 0])
# print(len(datalist))

## Get masked data
for root, dirs, files in os.walk(maskPath):
    if files == []:
        continue
    path1 = os.path.join(root, files[0])
    if len(files) > 1:
        path2 = os.path.join(root, files[1])
    datalist.append([path1, 1])
    datalist.append([path2, 1])

# print(len(datalist))

In [5]:
class faceMaskerDataset(Dataset):
    def __init__(self, filelist, transformation=None):
        self.transforms = transformation
        self.filelist = filelist
    
    def __getitem__(self, index):
        imagePath = self.filelist[index][0]
        label = self.filelist[index][1]
        img = Image.open(imagePath).resize([32,32])
        if self.transforms is not None:
            img = self.transforms(img)
        return (img, label)
    
    def __len__(self):
        return len(self.filelist)

transformation = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [6]:
dataset = faceMaskerDataset(datalist, transformation)
img, label = dataset[1]

## Conv Net

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1);
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
net = Net()

## Optimization Method

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
trainloader = torch.utils.data.DataLoader(dataset, batch_size=4, 
                                         shuffle=True, num_workers=2)
for epoch in range(Epoch):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 20 == 19:
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 20))
            running_loss = 0.0
print('Finished Training')

[1,    20] loss: 4.370
[1,    40] loss: 4.241
[1,    60] loss: 4.020
[1,    80] loss: 3.070
[1,   100] loss: 0.965
[1,   120] loss: 0.947
[1,   140] loss: 0.688
[1,   160] loss: 0.846
[1,   180] loss: 0.730
[1,   200] loss: 0.669
[1,   220] loss: 0.676
[1,   240] loss: 0.666
[1,   260] loss: 0.656
[1,   280] loss: 0.609
[1,   300] loss: 0.556
[1,   320] loss: 0.572
[1,   340] loss: 0.526
[1,   360] loss: 0.407
[1,   380] loss: 0.478
[1,   400] loss: 0.357
[1,   420] loss: 0.543
[1,   440] loss: 0.296
[1,   460] loss: 0.301
[2,    20] loss: 0.284
[2,    40] loss: 0.256
[2,    60] loss: 0.232
[2,    80] loss: 0.409
[2,   100] loss: 0.194
[2,   120] loss: 0.168
[2,   140] loss: 0.162
[2,   160] loss: 0.159
[2,   180] loss: 0.365
[2,   200] loss: 0.189
[2,   220] loss: 0.185
[2,   240] loss: 0.347
[2,   260] loss: 0.276
[2,   280] loss: 0.166
[2,   300] loss: 0.376
[2,   320] loss: 0.284
[2,   340] loss: 0.231
[2,   360] loss: 0.176
[2,   380] loss: 0.264
[2,   400] loss: 0.280
[2,   420] 